In [1]:
#Import packages
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from dmba import plotDecisionTree, classificationSummary, regressionSummary, liftChart, gainsChart

no display found. Using non-interactive Agg backend


In [2]:
soccer_df = pd.read_csv('/Users/jpmcelyea/SoccerML_Model/results.csv')

In [3]:
#soccer_df.dtypes

In [4]:
soccer_df['date'] = pd.to_datetime(soccer_df['date'])

In [5]:
soccer_df['year'] = pd.DatetimeIndex(soccer_df['date']).year

In [6]:
soccer_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1872
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False,1873
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,1874
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1875
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,1876


In [231]:
soccer_current = soccer_df[soccer_df['year'] >= 1930]

In [232]:
soccer_current['goal_diff'] = np.abs(soccer_current['home_score'] - soccer_current['away_score'])

<ipython-input-232-b4effac867eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soccer_current['goal_diff'] = np.abs(soccer_current['home_score'] - soccer_current['away_score'])


In [233]:
soccer_current['winner'] = np.where(soccer_current['home_score'] > soccer_current['away_score'], soccer_current['home_team'], np.where(soccer_current['home_score'] < soccer_current['away_score'], soccer_current['away_team'], 'Draw'))

<ipython-input-233-22fac4355087>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soccer_current['winner'] = np.where(soccer_current['home_score'] > soccer_current['away_score'], soccer_current['home_team'], np.where(soccer_current['home_score'] < soccer_current['away_score'], soccer_current['away_team'], 'Draw'))


In [234]:
soccer_current.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner
1267,1930-01-01,Spain,Czechoslovakia,1.0,0.0,Friendly,Barcelona,Spain,False,1930,1.0,Spain
1268,1930-01-12,Portugal,Czechoslovakia,1.0,0.0,Friendly,Lisbon,Portugal,False,1930,1.0,Portugal
1269,1930-01-26,Greece,Yugoslavia,2.0,1.0,Balkan Cup,Athens,Greece,False,1930,1.0,Greece
1270,1930-02-01,Northern Ireland,Wales,7.0,0.0,British Championship,Belfast,Northern Ireland,False,1930,7.0,Northern Ireland
1271,1930-02-09,Italy,Switzerland,4.0,2.0,Friendly,Rome,Italy,False,1930,2.0,Italy


In [235]:
euro_cup_teams = ['Turkey', 'Italy', 'Wales', 'Switzerland', 'Denmark', 'Finland', 'Belgium', 'Russia', 'Netherlands', 'Ukraine', 'Austria', 'North Macedonia', 'England', 'Croatia', 'Scotland', 'Czech Republic', 'Spain', 'Sweden', 'Poland', 'Slovakia', 'Hungary', 'Portugal', 'France', 'Germany']

In [236]:
soccer_current_teams = soccer_current[soccer_current['home_team'].isin(euro_cup_teams) & soccer_current['away_team'].isin(euro_cup_teams)]

In [237]:
soccer_current_teams['winner_home_advantage'] = np.where(soccer_current_teams['home_team'] == soccer_current_teams['winner'], 'yes', 'no')

<ipython-input-237-8aab3f0acb2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soccer_current_teams['winner_home_advantage'] = np.where(soccer_current_teams['home_team'] == soccer_current_teams['winner'], 'yes', 'no')


In [238]:
soccer_current_teams.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner,winner_home_advantage
1271,1930-02-09,Italy,Switzerland,4.0,2.0,Friendly,Rome,Italy,False,1930,2.0,Italy,yes
1274,1930-02-23,Portugal,France,2.0,0.0,Friendly,Porto,Portugal,False,1930,2.0,Portugal,yes
1275,1930-03-02,Germany,Italy,0.0,2.0,Friendly,Frankfurt am Main,Germany,False,1930,2.0,Italy,no
1277,1930-03-23,France,Switzerland,3.0,3.0,Friendly,Colombes,France,False,1930,0.0,Draw,no
1278,1930-04-05,England,Scotland,5.0,2.0,British Championship,London,England,False,1930,3.0,England,yes


In [239]:
soccer_current_teams[soccer_current_teams['neutral'] == False]['winner_home_advantage'].value_counts()

no     1919
yes    1691
Name: winner_home_advantage, dtype: int64

In [240]:
soccer_current_teams = soccer_current_teams.dropna()

In [241]:
soccer_current_teams.isna().sum()

date                     0
home_team                0
away_team                0
home_score               0
away_score               0
tournament               0
city                     0
country                  0
neutral                  0
year                     0
goal_diff                0
winner                   0
winner_home_advantage    0
dtype: int64

In [242]:
#soccer_current_teams[soccer_current_teams['home_team'].isin(['Belgium', 'Denmark']) & soccer_current_teams['away_team'].isin(['Belgium','Denmark'])]

In [243]:
#soccer_current_teams.tournament.value_counts()

In [244]:
#make column - friendly versus competitive
soccer_current_teams['type_tournament'] = np.where(soccer_current_teams['tournament'] == 'Friendly', 'Friendly', 'Competitive')

In [274]:
rankings_df = pd.read_csv('/Users/jpmcelyea/SoccerML_Model/rankings202102_euro.csv')

In [277]:
#soccer_current_teams['period'] = pd.cut(soccer_current_teams['year'], 5, labels=range(1, 6)).astype('category')

In [278]:
soccer_current_teams.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner,winner_home_advantage,type_tournament
1271,1930-02-09,Italy,Switzerland,4.0,2.0,Friendly,Rome,Italy,False,1930,2.0,Italy,yes,Friendly
1274,1930-02-23,Portugal,France,2.0,0.0,Friendly,Porto,Portugal,False,1930,2.0,Portugal,yes,Friendly
1275,1930-03-02,Germany,Italy,0.0,2.0,Friendly,Frankfurt am Main,Germany,False,1930,2.0,Italy,no,Friendly
1277,1930-03-23,France,Switzerland,3.0,3.0,Friendly,Colombes,France,False,1930,0.0,Draw,no,Friendly
1278,1930-04-05,England,Scotland,5.0,2.0,British Championship,London,England,False,1930,3.0,England,yes,Competitive


In [297]:
soccer_current_teams_total = soccer_current_teams.merge(rankings_df, left_on = 'home_team', right_on = 'Team', how = 'left')

In [298]:
soccer_current_teams_total = soccer_current_teams_total.rename({'Ranking': 'Home_Rank'}, axis=1)
soccer_current_teams_total = soccer_current_teams_total.drop(['Team'], axis = 1)

In [299]:
soccer_current_teams_total = soccer_current_teams_total.merge(rankings_df, left_on = 'away_team', right_on = 'Team', how = 'left')

In [300]:
soccer_current_teams_total = soccer_current_teams_total.rename({'Ranking': 'Away_Rank'}, axis=1)
soccer_current_teams_total = soccer_current_teams_total.drop(['Team'], axis = 1)

In [301]:
soccer_current_teams_total.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner,winner_home_advantage,type_tournament,Home_Rank,Away_Rank
0,1930-02-09,Italy,Switzerland,4.0,2.0,Friendly,Rome,Italy,False,1930,2.0,Italy,yes,Friendly,6,11
1,1930-02-23,Portugal,France,2.0,0.0,Friendly,Porto,Portugal,False,1930,2.0,Portugal,yes,Friendly,4,2
2,1930-03-02,Germany,Italy,0.0,2.0,Friendly,Frankfurt am Main,Germany,False,1930,2.0,Italy,no,Friendly,9,6
3,1930-03-23,France,Switzerland,3.0,3.0,Friendly,Colombes,France,False,1930,0.0,Draw,no,Friendly,2,11
4,1930-04-05,England,Scotland,5.0,2.0,British Championship,London,England,False,1930,3.0,England,yes,Competitive,3,28


In [302]:
soccer_current_teams_total['divide'] = np.where(soccer_current_teams_total['goal_diff'] >= 3, 'NotClose', 'Close')

In [303]:
soccer_current_teams_total['divide'].value_counts()

Close       3275
NotClose     720
Name: divide, dtype: int64

In [304]:
soccer_current_teams_total.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,goal_diff,winner,winner_home_advantage,type_tournament,Home_Rank,Away_Rank,divide
0,1930-02-09,Italy,Switzerland,4.0,2.0,Friendly,Rome,Italy,False,1930,2.0,Italy,yes,Friendly,6,11,Close
1,1930-02-23,Portugal,France,2.0,0.0,Friendly,Porto,Portugal,False,1930,2.0,Portugal,yes,Friendly,4,2,Close
2,1930-03-02,Germany,Italy,0.0,2.0,Friendly,Frankfurt am Main,Germany,False,1930,2.0,Italy,no,Friendly,9,6,Close
3,1930-03-23,France,Switzerland,3.0,3.0,Friendly,Colombes,France,False,1930,0.0,Draw,no,Friendly,2,11,Close
4,1930-04-05,England,Scotland,5.0,2.0,British Championship,London,England,False,1930,3.0,England,yes,Competitive,3,28,NotClose


In [305]:
soccer_current_teams_total = soccer_current_teams_total.reset_index(drop = True)

soccer_current_teams_total.loc[soccer_current_teams_total.winner == 'Draw', 'winner'] = 'Draw'
soccer_current_teams_total.loc[soccer_current_teams_total.winner == soccer_current_teams_total.home_team, 'winner'] = 'Home'
soccer_current_teams_total.loc[soccer_current_teams_total.winner == soccer_current_teams_total.away_team, 'winner'] = 'Away'


In [306]:
soccer_current_teams_total['winner'].tail()

3990    Home
3991    Home
3992    Away
3993    Away
3994    Draw
Name: winner, dtype: object

In [307]:
soccer_current_teams_total = soccer_current_teams_total.drop(['date', 'home_score', 'away_score', 'city', 'country', 'neutral', 'year', 'goal_diff', 'winner_home_advantage', 'tournament', 'divide'], axis = 1)

In [308]:
soccer_current_teams_total.head()

,home_team,away_team,winner,type_tournament,Home_Rank,Away_Rank
0,Italy,Switzerland,Home,Friendly,6,11
1,Portugal,France,Home,Friendly,4,2
2,Germany,Italy,Away,Friendly,9,6
3,France,Switzerland,Draw,Friendly,2,11
4,England,Scotland,Home,Competitive,3,28


In [309]:
predictors = ['home_team', 'away_team', 'type_tournament', 'Home_Rank', 'Away_Rank']

In [310]:
#Split into X and y variables
X = pd.get_dummies(soccer_current_teams_total[predictors], drop_first = True)
y = soccer_current_teams_total['winner']

In [311]:
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size = 0.4, random_state = 1)

In [369]:
rfModel = RandomForestClassifier(n_estimators=100, random_state=1)
rfModel.fit(train_X, train_y)

classificationSummary(valid_y, rfModel.predict(valid_X))

Confusion Matrix (Accuracy 0.4412)

       Prediction
Actual   0   1   2
     0 164  85 211
     1 110  72 212
     2 152 123 469


In [313]:
soccer_current_teams_total.head()

,home_team,away_team,winner,type_tournament,Home_Rank,Away_Rank
0,Italy,Switzerland,Home,Friendly,6,11
1,Portugal,France,Home,Friendly,4,2
2,Germany,Italy,Away,Friendly,9,6
3,France,Switzerland,Draw,Friendly,2,11
4,England,Scotland,Home,Competitive,3,28


In [316]:
pred_y = rfModel.predict(valid_X)
predProb_rf = rfModel.predict_proba(valid_X)

In [334]:
rfModel = pd.DataFrame({
    'rf_prob_home': predProb_rf[: ,2],
    'rf_prob_draw': predProb_rf[: ,1],
    'rf_prob_away': predProb_rf[: ,0],
    'rf_pred': pred_y,
    'actual': valid_y
    })

In [335]:
rfModel.head()

,rf_prob_home,rf_prob_draw,rf_prob_away,rf_pred,actual
200,0.627050,0.245536,0.127414,Home,Home
3254,0.018333,0.330079,0.651587,Away,Home
610,0.729977,0.202759,0.067264,Home,Home
1946,0.244909,0.712333,0.042758,Draw,Home
3559,0.534766,0.457146,0.008088,Home,Draw


In [326]:
predProb_rf

array([[0.12741389, 0.24553616, 0.62704995],
       [0.6515873 , 0.33007937, 0.01833333],
       [0.06726433, 0.20275916, 0.72997651],
       ...,
       [0.20431945, 0.22929772, 0.56638283],
       [0.60377686, 0.19731162, 0.19891152],
       [0.03      , 0.06      , 0.91      ]])

In [462]:
euros_cup_df = pd.read_csv('/Users/jpmcelyea/SoccerML_Model/euro_2020_matches.csv')

In [464]:
euros_cup_df = euros_cup_df[euros_cup_df['Round Number'] == '1']

In [465]:
euros_cup_df = euros_cup_df.rename(columns={"Home Team": "home_team", "Away Team": "away_team"})

In [466]:
euros_cup_df['type_tournament'] = 'Competitive'

In [467]:
euros_cup_df = euros_cup_df.drop(['Date', 'Location', 'Group', 'Result', 'Match Number', 'Round Number'], axis = 1)

In [468]:
euros_cup_df = euros_cup_df.merge(rankings_df, left_on = 'home_team', right_on = 'Team', how = 'left')
euros_cup_df = euros_cup_df.rename({'Ranking': 'Home_Rank'}, axis=1)
euros_cup_df = euros_cup_df.drop(['Team'], axis = 1)

In [469]:
euros_cup_df = euros_cup_df.merge(rankings_df, left_on = 'away_team', right_on = 'Team', how = 'left')
euros_cup_df = euros_cup_df.rename({'Ranking': 'Away_Rank'}, axis=1)
euros_cup_df = euros_cup_df.drop(['Team'], axis = 1)

In [470]:
euros_cup_df.isna().sum()

home_team          0
away_team          0
type_tournament    0
Home_Rank          0
Away_Rank          0
dtype: int64

In [471]:
euros_cup_df['Home_Rank'] = euros_cup_df['Home_Rank'].astype('int64')
euros_cup_df['Away_Rank'] = euros_cup_df['Away_Rank'].astype('int64')

In [472]:
euros_cup_df.head()

,home_team,away_team,type_tournament,Home_Rank,Away_Rank
0,Turkey,Italy,Competitive,18,6
1,Wales,Switzerland,Competitive,12,11
2,Denmark,Finland,Competitive,8,30
3,Belgium,Russia,Competitive,1,21
4,England,Croatia,Competitive,3,7


In [447]:
soccer_current_teams_total.head()

,home_team,away_team,winner,type_tournament,Home_Rank,Away_Rank
0,Italy,Switzerland,Home,Friendly,6,11
1,Portugal,France,Home,Friendly,4,2
2,Germany,Italy,Away,Friendly,9,6
3,France,Switzerland,Draw,Friendly,2,11
4,England,Scotland,Home,Competitive,3,28


In [475]:
euros_cup_df = pd.get_dummies(euros_cup_df, drop_first = True)

In [476]:
euros_cup_df = euros_cup_df.reindex(labels = valid_X.columns, axis = 1, fill_value = 0)

In [453]:
pred_euro = rfModel.predict(euros_cup_df)
predProb_euro = rfModel.predict_proba(euros_cup_df)

In [455]:
rfModel_euro = pd.DataFrame({
    'rf_prob_home': predProb_euro[: ,2],
    'rf_prob_draw': predProb_euro[: ,1],
    'rf_prob_away': predProb_euro[: ,0],
    'rf_pred': pred_euro
    })


In [457]:
rfModel_euro

,rf_prob_home,rf_prob_draw,rf_prob_away,rf_pred
0,0.377770,0.140294,0.481937,Away
1,0.438730,0.191255,0.370015,Home
2,0.661358,0.224000,0.114643,Home
3,0.610585,0.244333,0.145082,Home
4,0.507821,0.224500,0.267679,Home
5,0.742846,0.150827,0.106327,Home
6,0.574806,0.264167,0.161028,Home
7,0.478111,0.200921,0.320968,Home
8,0.376763,0.307776,0.315461,Home
9,0.609559,0.250587,0.139854,Home


In [477]:
euros_cup_df

,Home_Rank,Away_Rank,home_team_Belgium,home_team_Croatia,home_team_Czech Republic,home_team_Denmark,home_team_England,home_team_Finland,home_team_France,home_team_Germany,...,away_team_Russia,away_team_Scotland,away_team_Slovakia,away_team_Spain,away_team_Sweden,away_team_Switzerland,away_team_Turkey,away_team_Ukraine,away_team_Wales,type_tournament_Friendly
0,18,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,8,30,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,21,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,3,7,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,15,34,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,10,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,28,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,13,19,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,5,14,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
